In [6]:
# -----------------------------------------------------------
# Faza 2:
# Implementacija flair NER
# -----------------------------------------------------------


# import nltk
# from nltk import pos_tag, word_tokenize
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('gutenberg')
# import re
# import string
# from itertools import combinations
# from collections import Counter


from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from flair.models import SequenceTagger
from flair.data import Sentence
import operator


In [7]:
# Use flair named entity recognition
tagger = SequenceTagger.load('ner')     # ner, ner-ontonotes, ...

2022-05-19 10:46:01,741 loading file /Users/matijagercer/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-05-19 10:46:06,352 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [12]:
def read_text(path):
    with open(path, encoding="utf-8") as f:
        text = f.read()
        text = text.replace('\r', ' ').replace('\n', ' ')\
            .replace("’", "'").replace("\"", "").replace("”", "").replace("“", "")
    return text

def flair_NER(book):
    """
    flair_NER vrne seznam, v katerem so shranjene prepoznane identitete glede na posamezni stavek
    :param book: str
    :return: entity_dict (seznam slovarjev kot npr. [name, tag, start_pos, stop_pos, line_num. token_num])
    """

    # 00 Pretvori knjigo v stavke:
    sent = sent_tokenize(book)

    # 01 NER model
    entity_dict = []
    for line_num, line in enumerate(tqdm(sent)):

        # 01a Predict:
        sentence = Sentence(line)
        tagger.predict(sentence)

        # 01b Vmesni izpis (brez lokacije):
        #print(sentence.to_tagged_string())
        #print(sentence.get_spans('ner'))

        # 01c Ekstrakcija podatka iz stavka:
        for entity in sentence.get_spans('ner'):        # veliko različnih flair - nerov
            name = entity.text

            # str location
            # start_pos = entity.start_position    # št. str
            # stop_pos = entity.end_position      # št. str

            # token location
            tmp_flag = True
            for token in entity:
                if tmp_flag:
                    start_pos = token.idx -1
                    stop_pos = token.idx
                else:
                    stop_pos = token.idx
                tmp_flag = False

            tag = entity.get_label("ner").value             # tag = entity.tag
            conf_score = entity.get_label("ner").score      # conf_score = entity.score

            info_dict = {}
            info_dict["name"] = name
            info_dict["tag"] = tag
            info_dict["start_pos"] = start_pos
            info_dict["stop_pos"] = stop_pos
            info_dict["line_num"] = line_num

            entity_dict.append(info_dict)

    return entity_dict

def get_names_from_NER(entity_dict):
    """
    get_names_from_NER sprejme entity_dict in vrne urejen seznam terk ("ime", št_zaznano)
    :param entity_dict: dict (seznam dictov)
    :return: unique_names: list
    """
    unique_names = {}

    for entity in entity_dict:
        if entity["tag"] == "PER":
            if entity["name"] not in unique_names:
                unique_names[entity["name"]] = 1
            else:
                unique_names[entity["name"]] += 1
    unique_names = sorted(unique_names.items(), key=operator.itemgetter(1),reverse=True)

    return unique_names

In [ ]:
book = read_text('../data/books/ASongOfIceAndFire/AGOT/chapters/Bran_1_1.txt')
entity_dict = flair_NER(book)
unique_names = get_names_from_NER(entity_dict)

for (name, num) in unique_names:
    if True: #num > 1:
        print(name, num)

In [ ]:
def get_named_entities(sentences):
    threshold = 0.0001
    named_entities = []
    named_entities_thresholded = []
    for s in sentences:
        named_entities += NER(s)

    named_entities = [ne for ne in named_entities if ne not in common_english_words]
    named_entities = Counter(named_entities)
    for ne in named_entities:
        if named_entities[ne] >= threshold * len(sentences):
            named_entities_thresholded.append(ne)

    return named_entities_thresholded

In [ ]:
def get_main_characters(book, characters, number_of_characters):
    cv = CountVectorizer(vocabulary=characters, stop_words='english')
    freq = cv.fit_transform([book.lower()])
    freq = pd.DataFrame(freq.toarray(), columns=cv.get_feature_names())
    freq = freq.T
    freq = freq.sort_values(by=0, ascending=False)
    freq = freq[0:number_of_characters]

    characters = list(freq.index)
    return freq, characters

In [ ]:
def plot_graph(name_list, name_frequency, matrix, plt_name, mode, path=''):

    label = {i: i for i in name_list}
    edge_list = []
    shape = matrix.shape[0]
    triangle = list(zip(*np.where(np.triu(np.ones([shape, shape])) == 0)))
    normalized_matrix = matrix / np.max(np.abs(matrix))
    normalized_frequency = np.array(name_frequency) / np.max(name_frequency)
    weight = np.log(np.abs(1000 * normalized_matrix) + 1) * 0.7
    color = 2000 * normalized_matrix
    for i in triangle:
        edge_list.append((name_list[i[0]], name_list[i[1]], {'weight': weight[i], 'color': color[i]}))
    plt.figure(figsize=(12, 12))
    G = nx.Graph()
    G.add_nodes_from(name_list)
    G.add_edges_from(edge_list)
    edges = G.edges()
    weights = [G[u][v]['weight'] for u, v in edges]
    colors = [G[u][v]['color'] for u, v in edges]

    
    nx.draw_random(G, node_color='#A0CBE2', node_size=normalized_frequency * 2000,
            linewidths=10, labels=label, edge_color=colors, with_labels=True,
            width=weights, edge_vmin=-1000, edge_vmax=1000)


    plt.savefig(path + plt_name + '.png')

In [ ]:
nlp = spacy.load('en_core_web_md')
common_english_words = get_common_english_words('../data/common_english_words.txt')
book = read_text('../data/books/ASongOfIceAndFire/GoT1_long.txt')
sentences = nltk.sent_tokenize(book)
characters = get_named_entities(sentences)
freq, main_characters = get_main_characters(book, characters, number_of_characters=25)
sentiment_mtx, _ = get_mtx(sentences, main_characters)

In [ ]:
temp = freq.values
temp = temp.tolist()
freq = [item for sublist in temp for item in sublist]

In [ ]:
plot_graph(main_characters, freq, sentiment_mtx, "test" + ' sentiment graph', 'sentiment')